In [5]:
import collections
import pathlib
import json

In [6]:
def load_results(results_dir):
    results = collections.defaultdict(list)
    for results_dir_i in sorted(results_dir.iterdir(), key=lambda k: int(k.name)):
        print(results_dir_i)
        for result_path in results_dir_i.iterdir():
            with open(result_path) as f:
                result = json.load(f)
            # assert result.get('pov_answer_correct')
            results[result_path.name].append(result)
    return results

In [7]:
results = load_results(pathlib.Path('results'))

results/01
results/02


results/03
results/04
results/05


results/06
results/07
results/08


results/09
results/10
results/11


results/12
results/13
results/14


results/15
results/16


In [8]:
results[list(results.keys())[0]][0].keys()

dict_keys(['flag', 'interaction', 'signal', 'pov_answer', 'pov_answer_correct'])

In [9]:
for i, challenge in enumerate(sorted(results)):
    print(i, '\t', challenge)
    
    challenge_results = results[challenge]
    interaction_lengths = [len(data.get('interaction', [])) for data in challenge_results]
    signals = [data.get('signal', 0) for data in challenge_results]
    correct = [data.get('pov_answer_correct') for data in challenge_results]

    print('\t', 'interaction lengths:', collections.Counter(interaction_lengths))
    print('\t', '\t', interaction_lengths)
    if any(signals):
        print('\t', 'signals:', collections.Counter(signals))
        print('\t', '\t', signals)
    if not all(correct):
        print('\t', 'correct:', correct)
    print()

0 	 CROMU_00051_pov_1.pov
	 interaction lengths: Counter({85: 8, 81: 3, 84: 2, 82: 2, 83: 1})
	 	 [84, 85, 83, 81, 85, 82, 85, 81, 85, 82, 85, 85, 85, 85, 81, 84]

1 	 CROMU_00051_pov_2.pov
	 interaction lengths: Counter({188: 10, 187: 3, 185: 2, 184: 1})
	 	 [185, 185, 187, 187, 188, 188, 188, 187, 188, 188, 188, 188, 188, 188, 184, 188]

2 	 CROMU_00054_pov
	 interaction lengths: Counter({120: 6, 119: 5, 117: 3, 121: 1, 116: 1})
	 	 [117, 121, 120, 119, 119, 117, 116, 119, 119, 120, 119, 120, 120, 120, 117, 120]

3 	 CROMU_00057_pov
	 interaction lengths: Counter({23: 16})
	 	 [23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]
	 signals: Counter({11: 16})
	 	 [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]

4 	 CROMU_00058_pov
	 interaction lengths: Counter({20: 16})
	 	 [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

5 	 CROMU_00077_pov
	 interaction lengths: Counter({16: 16})
	 	 [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1

In [10]:
# Merge consecutive reads / writes

for i, challenge in enumerate(sorted(results)):
    print(i, '\t', challenge)
    
    challenge_results = results[challenge]
    interaction_lengths = []
    first_elements = []
    last_elements = []
    for data in challenge_results:
        length = 0
        prev = None
        for current, _, _ in data.get('interaction', []):
            if current != prev:
                length += 1
            prev = current
        interaction_lengths.append(length)
        last_elements.append(prev)
        
    print('\t', 'interaction lengths:', interaction_lengths)
    print('\t', 'same order:', len(set(interaction_lengths)) == 1 and len(set(last_elements)) == 1)
    print()

0 	 CROMU_00051_pov_1.pov
	 interaction lengths: [13, 13, 13, 12, 13, 13, 13, 12, 13, 13, 13, 13, 13, 13, 12, 13]
	 same order: False

1 	 CROMU_00051_pov_2.pov
	 interaction lengths: [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 39]
	 same order: False

2 	 CROMU_00054_pov
	 interaction lengths: [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]
	 same order: True

3 	 CROMU_00057_pov
	 interaction lengths: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
	 same order: True

4 	 CROMU_00058_pov
	 interaction lengths: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
	 same order: True

5 	 CROMU_00077_pov
	 interaction lengths: [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
	 same order: True

6 	 CROMU_00078_pov
	 interaction lengths: [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
	 same order: True

7 	 CROMU_00079_pov
	 interaction lengths: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
	 same order: True

8 	 CROMU_

In [31]:
# Do PoVs send the exact same data?
# Do PoVs interact in the same order

content_positive = []
content_negative = []
content_min_positive = []
content_min_negative = []

order_positive = []
order_negative = []
order_min_positive = []
order_min_negative = []

for i, challenge in enumerate(sorted(results)):
    print(i, '\t', challenge)
    
    challenge_results = results[challenge]
    
    data_reads = []
    for result in challenge_results:
        data_read = ''
        for syscall, length, data in result.get('interaction', []):
            if syscall == 'read':
                data_read += data
        data_reads.append(data_read)
        
    directions = []
    for result in challenge_results:
        direction = ''
        for syscall, length, data in result.get('interaction', []):
            direction += syscall[0]
        directions.append(direction)
        
        
    # if not all(len(e) == len(data_reads[0]) for e in data_reads):
    #     continue
        
    min_length = min(len(e) for e in data_reads)
    min_data_reads = [e[:min_length] for e in data_reads]
    
    matching = [data_read == data_reads[0] for data_read in data_reads]
    print('\t', 'content matching:', all(matching))
    
    min_matching = [data_read == min_data_reads[0] for data_read in min_data_reads]
    print('\t', 'content min matching:', all(min_matching))

    if all(matching):
        content_positive.append(challenge)
    else:
        content_negative.append(challenge)
        
    if all(min_matching):
        content_min_positive.append(challenge)
    else:
        content_min_negative.append(challenge)

    min_length = min(len(e) for e in directions)
    min_directions = [e[:min_length] for e in directions]
    
    matching = [direction == directions[0] for direction in directions]
    print('\t', 'order matching:', all(matching))
    
    min_matching = [direction == min_directions[0] for direction in min_directions]
    print('\t', 'order min matching:', all(min_matching))

    if all(matching):
        order_positive.append(challenge)
    else:
        order_negative.append(challenge)
        
    if all(min_matching):
        order_min_positive.append(challenge)
    else:
        order_min_negative.append(challenge)        
        
    print()

print('content positive:', len(content_positive))
print('content negative:', len(content_negative))
print('content min positive:', len(content_min_positive))
print('content min negative:', len(content_min_negative))

print('order positive:', len(order_positive))
print('order negative:', len(order_negative))
print('order min positive:', len(order_min_positive))
print('order min negative:', len(order_min_negative))

print('total negative:', len(set(content_negative) & set(order_negative)))
print('total min negative:', len(set(content_min_negative) & set(order_min_negative)))

0 	 CROMU_00051_pov_1.pov
	 content matching: True
	 content min matching: True
	 order matching: False
	 order min matching: True

1 	 CROMU_00051_pov_2.pov
	 content matching: True
	 content min matching: True
	 order matching: False
	 order min matching: True

2 	 CROMU_00054_pov
	 content matching: True
	 content min matching: True
	 order matching: False
	 order min matching: False

3 	 CROMU_00057_pov
	 content matching: True
	 content min matching: True
	 order matching: True
	 order min matching: True

4 	 CROMU_00058_pov
	 content matching: True
	 content min matching: True
	 order matching: True
	 order min matching: True

5 	 CROMU_00077_pov
	 content matching: True
	 content min matching: True
	 order matching: True
	 order min matching: True

6 	 CROMU_00078_pov
	 content matching: True
	 content min matching: True
	 order matching: False
	 order min matching: False

7 	 CROMU_00079_pov
	 content matching: True
	 content min matching: True
	 order matching: True
	 order mi

In [34]:
for challenge in set(content_min_negative) & set(order_min_negative):
    print(challenge)

    challenge_results = results[challenge]

    for result in challenge_results:        
        direction = ''
        for syscall, length, data in result.get('interaction', []):
            direction += syscall[0]
        print(direction)

    print()
    

NRFIN_00046_POV_00001.pov
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr
rwrrrrwwrwrrrrwwrwrrrwwrwrrwwr

KPRCA_00112_pov
rwwwrrwwrw
rwwwrrwwrw
rwwwrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrrwwrw
rwwwrwwrw

KPRCA_00071_pov_1.pov
wwwrwwwwwwwwwwwwwwwwwwwwwwrwwwwwwwwwwwwwwwwwwwwwwwwwwrwrwwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwwwwwwwwwww
wwwrwwwwwwwwwwwwwwwwwwwwwwrwwwwwwwwwwwwwwwwwwwwwwwwwwrwrwwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrwwrw